# ***Project 3: IMBD Queries***

In [1]:
# Import libraries:

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# Load Data:

basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

# DataFrame: 

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


# **Preprocessing**

## **Filtering/Cleaning Steps:**

### **Title Basics:**

1. Replace "\N" with np.nan

2. Eliminate movies that are null for runtimeMinutes

3. Eliminate movies that are null for genre keep only titleType==Movie

4. keep startYear 2000-2022

5. Eliminate movies that include "Documentary" in genre (see tip below)

In [3]:
# Replace "\N" with np.nan

basics.replace({'\\N':np.nan}, inplace =True) 

In [4]:
basics['runtimeMinutes'].values

array(['1', '5', '4', ..., nan, '27', '10'], dtype=object)

In [5]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9044976 entries, 0 to 9044975
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 621.1+ MB


In [14]:
# Eliminate movies that are null for runtimeMinutes

basics.dropna(subset=['runtimeMinutes'], inplace=True)
basics['runtimeMinutes'].value_counts()

90     25979
80     11590
60     11180
85     10870
95     10361
       ...  
575        1
663        1
371        1
665        1
990        1
Name: runtimeMinutes, Length: 490, dtype: int64

In [6]:
basics['runtimeMinutes'].isna().sum()

6609301

In [7]:
# Eliminate movies that are null for genre:

basics.dropna(subset=['genres'], inplace=True)

basics['genres'].value_counts()

Drama                              1022865
Comedy                              621209
Talk-Show                           539636
News                                452606
Documentary                         434991
                                    ...   
Documentary,Sci-Fi,Thriller              1
History,Horror,News                      1
Music,Reality-TV,Sport                   1
Adventure,Film-Noir                      1
Documentary,Reality-TV,Thriller          1
Name: genres, Length: 2315, dtype: int64

In [8]:
# Keep only titleType==Movie: 

movie = basics['titleType'] == 'movie'

basics = basics[movie]

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,NaN,Drama
625,tt0000630,movie,Hamlet,Amleto,0,1908,NaN,NaN,Drama
668,tt0000675,movie,Don Quijote,Don Quijote,0,1908,NaN,NaN,Drama


In [9]:
basics['startYear'].value_counts()

2018    17680
2017    17542
2019    17178
2016    17067
2021    16580
        ...  
1903        3
2029        3
1904        2
1897        1
1896        1
Name: startYear, Length: 134, dtype: int64

In [11]:
# keep startYear 2000-2022:

# Conversion to date time 

basics['startYear'] =pd.to_datetime(basics['startYear'])


start_date = '2000'
end_date = '2023'

daterange = (basics['startYear'] >= start_date) & (basics['startYear'] <= end_date)

basics = basics.loc[daterange]

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
11636,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019-01-01,NaN,NaN,"Action,Crime"
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021-01-01,NaN,133,Documentary
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001-01-01,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020-01-01,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018-01-01,NaN,122,Drama


In [12]:
# Eliminate movies that include "Documentary" in genre (see tip below):

documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~documentary]

basics.head()

basics['genres'].value_counts()

Drama                       46672
Comedy                      17475
Horror                       7964
Comedy,Drama                 6813
Thriller                     5936
                            ...  
History,Romance,Thriller        1
Adventure,History,Music         1
Adventure,Romance,Sport         1
Action,Music,Sci-Fi             1
Action,Crime,Short              1
Name: genres, Length: 1011, dtype: int64

## **Filtering/Cleaning Steps:**

### **AKAs:**

1. keep only US entries.

2. Replace "\N" with np.nan

In [13]:
# Load Data:

akas_url ="https://datasets.imdbws.com/title.akas.tsv.gz"

# DataFrame: 

akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

# akas.head(5)

ParserError: Error tokenizing data. C error: out of memory

In [14]:
# Keep only US entires:

us = akas['region'] == 'US'

akas = akas[us]

akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [15]:
# Replace "\N" with np.nan: 

akas.replace({'\\N':np.nan}, inplace =True) 

akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


## **Filtering/Cleaning Steps:**

### **Ratings:**

1. Replace "\N" with np.nan (if any)

In [16]:
# Load Data:

rating_url ="https://datasets.imdbws.com/title.ratings.tsv.gz"

# DataFrame: 
ratings = pd.read_csv(rating_url, sep='\t', low_memory=False)

ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1892
1,tt0000002,5.9,252
2,tt0000003,6.5,1685
3,tt0000004,5.7,165
4,tt0000005,6.2,2501


In [17]:
# Replace "\N" with np.nan: 

ratings.replace({'\\N':np.nan}, inplace =True) 

ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1892
1,tt0000002,5.9,252
2,tt0000003,6.5,1685
3,tt0000004,5.7,165
4,tt0000005,6.2,2501


# **Filtering one DataFrame based on another**

Filter the basics df to only include the movies that are present in your filter akas dataframe. This is how you will ultimately be able to filter the movies by region being in the US.  

In [14]:
# Filter the basics table down to only include the US by using the filter akas dataframe:

ustitles = basics['tconst'].isin(akas['titleId'])
basics[ustitles]

NameError: name 'akas' is not defined

# **Creating a "Data" folder**

In [19]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

[]

In [1]:
# Save current dataframe to file:

basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

NameError: name 'basics' is not defined

In [21]:
# Save current dataframe to file:

akas.to_csv("Data/akas.csv.gz",compression='gzip',index=False)

In [22]:
# Save current dataframe to file:

ratings.to_csv("Data/ratings.csv.gz",compression='gzip',index=False)